In [2]:
import os
import json
import pickle

In [ ]:
import matplotlib.pyplot as plt
# Start by importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt2
import seaborn as sns
import datetime
import pickle
import warnings
warnings.filterwarnings('ignore')
from pybaseball import schedule_and_record
from pybaseball import statcast
from pybaseball import playerid_lookup
from pybaseball import statcast_pitcher
from pybaseball import team_batting
from pybaseball import team_pitching
from pybaseball import pitching_stats_bref
from pybaseball import batting_stats_bref
from pybaseball import batting_stats_range
from pybaseball import pitching_stats_range
from pybaseball import statcast_single_game
from pybaseball import playerid_reverse_lookup
from pybaseball import playerid_reverse_lookup
import os
import numbers
import math
import json

In [4]:
path = "../data/seasons/"
files = os.listdir(path)
files.sort()
batters = []
pitchers = []
for file in files:
    print(file)
    pickle_in = open(f"{path}/{file}","rb")
    every_pitch = pickle.load(pickle_in)
    every_pitch = every_pitch.iloc[::-1]
    for index, row in every_pitch.iterrows(): 
        batter = str(row.batter)
        if batter not in batters:
            batters.append(str(row.batter))
        pitcher = str(row.pitcher)
        if pitcher not in pitchers:
            pitchers.append(str(row.pitcher))
       
with open(f'pitchers_id.json', 'w', encoding='utf-8') as f:
    json.dump(pitcher, f, ensure_ascii=True, indent=4, allow_nan=True)
    f.close()

with open(f'batters_id.json', 'w', encoding='utf-8') as f:
    f.writelines(batters)
    f.close()

every_pitch_2003.pickle
every_pitch_2004.pickle
every_pitch_2005.pickle
every_pitch_2006.pickle
every_pitch_2007.pickle
every_pitch_2008.pickle
every_pitch_2009.pickle


UnpicklingError: invalid load key, '\xdd'.

In [ ]:

players = {}
with open(f'pitchers_id.json', 'r', encoding='utf-8') as f:
    pitchers = json.load(f)
    with open(f'batters_id.json', 'r', encoding='utf-8') as f:
        batters = json.load(f)
        count = 0
        for pitcher in pitchers:
            players.append({f"{pitcher}":playerid_reverse_lookup(pitcher)})
        for batter in batters:
             players.append({f"{batter}":playerid_reverse_lookup(batter)})

In [ ]:
path = "../seasons/"
files = os.listdir(path)
files.sort()
# stats_file = "../mlb_data/mlb.json"
baseball = []
lines = []
test = []
# with open(stats_file, 'r') as f:
    # stats = json.load(f)
season_count = 0
for file in files:
    print(file)
    pickle_in = open(f"{path}/{file}","rb")
    every_pitch = pickle.load(pickle_in)
    every_pitch = every_pitch.iloc[::-1]
    season = []
    at_bat = []
    pre_batter = ""
    for index, row in every_pitch.iterrows():
        current_batter = str(row.batter)
        if current_batter not in pre_batter:
            at_bat = []
        pre_batter = current_batter
        input = f"{row.inning_topbot} of the {row.inning} inning with {row.outs_when_up} outs\n"
        pitch = f" a {int(row.release_speed) if isinstance(row.release_speed, float) else 0} miles per hour {row.pitch_name}"
        response = f"{row.pitcher} throws a"
        if row.type == 'S':
            response = f"{response} strike, "
            if not isinstance(row.events, str):
                at_bat.append(response)
            if isinstance(row.events, str):
                # response = ""
                event = row.events 
                event = event.replace("_", " ")                   
                response =  f"{response} and {row.batter} {event}"
                        
                at_bat.append(response)
                output = ""
                for pitch in at_bat:
                    output =f"{output}{pitch}\n"
                output = f"{output} event: {event}"                          
                baseball.append(
                    {
                    'instruction':f"what is the outcome of pitcher {row.pitcher} pitching to batter {row.batter}",
                    'input':input,
                    "output":output
                    }
                    )
                lines.append(f"<s>##INSTRUCTION {f'what is the outcome of pitcher {row.pitcher} pitching to batter {row.batter}'} ###INPUT {input} ###OUTPUT {output}</s>\n") 
        elif row.type == 'B':
            response = f"{response} ball,"
            if not isinstance(row.events, str):
                at_bat.append(response)
            if isinstance(row.events, str):
                # response = ""
                event = row.events 
                event = event.replace("_", " ")                   
                response =  f"{response} and {row.batter} {event}s"
                        
                at_bat.append(response)
                output = ""
                for pitch in at_bat:
                    output =f"{output}{pitch}\n"
                output = f"{output} event: {event}"                          
                baseball.append(
                    {
                    'instruction':f"what is the outcome of pitcher {row.pitcher} pitching to batter {row.batter}",
                    'input':input,
                    "output":output
                    }
                    )
                lines.append(f"<s>##INSTRUCTION {f'what is the outcome of pitcher {row.pitcher} pitching to batter {row.batter}'} ###INPUT {input} ###OUTPUT {output}</s>\n") 
        elif row.type == 'X': 
            if isinstance(row.events, str):
                # response = ""
                event = row.events 
                event = event.replace("_", " ")
                # score = "" if (row.post_bat_score - row.bat_score) < 1 else f" and {(row.post_bat_score - row.bat_score)} runs scores"
                if 'single' in event or 'double' in event or 'triple' in event or 'home run' in event or "sac fly" in event or "sac" in event:
                            response = f"{response} and {row.batter} hits a {event}"
                elif 'grounded' not in event and ('field out' in event or 'double play' in event or 'force out' in event):
                    response =  f"{response} and {row.batter} hits into a {event}"
                else:
                    response =  f"{response} and {row.batter} {event}"
                        
                at_bat.append(response)
                output = ""
                for pitch in at_bat:
                    output =f"{output}{pitch}\n"
                output = f"{output} event: {event}"                          
                baseball.append(
                    {
                    'instruction':f"what is the outcome of pitcher {row.pitcher} pitching to batter {row.batter}",
                    'input':input,
                    "output":output
                    }
                    )
                lines.append(f"<s>##INSTRUCTION {f'what is the outcome of pitcher {row.pitcher} pitching to batter {row.batter}'} ###INPUT {input} ###OUTPUT {output}</s>\n") 
            else:
                if isinstance(row.events, str):
                # response = ""
                    event = row.events 
                    event = event.replace("_", " ")
                    response =  f"{response} and {row.batter} {event}"                 
                    at_bat.append(response)
                    output = ""
                    for pitch in at_bat:
                        output =f"{output}{pitch}\n"
                    output = f"{output} event: {event}"                       
                    baseball.append(
                        {
                        'instruction':f"what is the outcome of pitcher {row.pitcher} pitching to batter {row.batter}",
                        'input':input,
                        "output":output
                        }
                        )
                    lines.append(f"<s>##INSTRUCTION {f'what is the outcome of pitcher {row.pitcher} pitching to batter {row.batter}'} ###INPUT {input} ###OUTPUT {output}</s>\n")     
    season_count+=1

with open(f'baseball_10_01.json', 'w', encoding='utf-8') as f:
    json.dump(baseball, f, ensure_ascii=True, indent=4, allow_nan=True)
    f.close()

with open(f'baseball_10_01.txt', 'w', encoding='utf-8') as f:
    f.writelines(lines)
    f.close()

# with open(f'testing_short.json', 'w', encoding='utf-8') as f:
#     json.dump(testing, f, ensure_ascii=True, indent=4, allow_nan=True)
#     f.close()

# with open(f'batters_short.json', 'w', encoding='utf-8') as f:
#     json.dump(batters, f, ensure_ascii=True, indent=4, allow_nan=True)
#     f.close()

# with open(f'pitchers_short.json', 'w', encoding='utf-8') as f:
#     json.dump(pitchers, f, ensure_ascii=True, indent=4, allow_nan=True)
#     f.close()